In [21]:
import pandas as pd
from pandas import Series, DataFrame

In [22]:
#get the train and test dataset
train_df = pd.read_csv('D:/kaggleCompetition/TitanicDataSet/train.csv',header=0)  
test_df = pd.read_csv('D:/kaggleCompetition/TitanicDataSet/test.csv',header=0)  

In [23]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
train_df.shape,test_df.shape

((891, 12), (418, 11))

In [25]:
# merge the two DataFrames into one  
merge_df = pd.concat([train_df,test_df])  
merge_df = merge_df.reset_index()  
merge_df = merge_df.drop('index', axis=1)  
merge_df = merge_df.reindex_axis(train_df.columns, axis=1)  

In [26]:
merge_df.shape

(1309, 12)

In [27]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [28]:
merge_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [29]:
""" cope with the missing data """
# 1.'Cabin', seems useless, too much missing, two ways to handle it  
# (1) drop this feature  
merge_df = merge_df.drop('Cabin', axis=1)  
# (2) assign a value represent it's missing  
#df[df['Cabin'].isnull()] = 'missing' 

In [30]:
# 2. 'Age', a important feature, we can't just drop it, two ways to handle it  
# (1) fill it with median  
merge_df['Age'][merge_df['Age'].isnull()] = merge_df['Age'].median()  
# (2) use a simple model to predict it, after everything else is fixed.In this case,  
#  the accuracy of the experiment consequence is too low,so we give up this method.  
  
# 3. 'Fare', only one miss, just use the median  
merge_df['Fare'][merge_df['Fare'].isnull()] = merge_df['Fare'].median()  
  
# 4.'Embarked', two  miss, use the most frequency word  
merge_df['Embarked'][merge_df['Embarked'].isnull()] = merge_df['Embarked'].mode().values  

D:\DevelopSoftware\AnacondaSoft\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
D:\DevelopSoftware\AnacondaSoft\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\DevelopSoftware\AnacondaSoft\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
""" feature engineering """   
# 1. 'Sex', factorizing it  
merge_df['Sex'] = merge_df.factorize(merge_df['Sex'])[0]  
# 2. 'Name', we can drag some title from the name, for simplicity, I just drop it   
merge_df = merge_df.drop('Name', axis=1)  
# 3. 'Ticket', for simlicity, just drop it  
merge_df = merge_df.drop('Ticket', axis=1)   
# 4. 'Embarked', factorizing it    
merge_df['Embarked'] = merge_df.factorize(merge_df['Embarked'])[0]   
# 5. 'SibSp', 'Parch', combine them to get a new feature, 'Famliy_size'  
merge_df['Family_size'] = merge_df['SibSp']+merge_df['Parch']  

AttributeError: 'DataFrame' object has no attribute 'factorize'